In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("earthquake_1995-2023.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      1000 non-null   object 
 1   magnitude  1000 non-null   float64
 2   date_time  1000 non-null   object 
 3   cdi        1000 non-null   int64  
 4   mmi        1000 non-null   int64  
 5   alert      449 non-null    object 
 6   tsunami    1000 non-null   int64  
 7   sig        1000 non-null   int64  
 8   net        1000 non-null   object 
 9   nst        1000 non-null   int64  
 10  dmin       1000 non-null   float64
 11  gap        1000 non-null   float64
 12  magType    1000 non-null   object 
 13  depth      1000 non-null   float64
 14  latitude   1000 non-null   float64
 15  longitude  1000 non-null   float64
 16  location   994 non-null    object 
 17  continent  284 non-null    object 
 18  country    651 non-null    object 
dtypes: float64(6), int64(5), object(8)
memory usage: 

In [4]:
df.head()

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
0,"M 6.5 - 42 km W of Sola, Vanuatu",6.5,16-08-2023 12:47,7,4,green,0,657,us,114,7.177000,25.0,mww,192.955,-13.8814,167.1580,"Sola, Vanuatu",NaN,Vanuatu
1,"M 6.5 - 43 km S of Intipucá, El Salvador",6.5,19-07-2023 00:22,8,6,yellow,0,775,us,92,0.679000,40.0,mww,69.727,12.8140,-88.1265,"Intipucá, El Salvador",NaN,NaN
2,"M 6.6 - 25 km ESE of Loncopué, Argentina",6.6,17-07-2023 03:05,7,5,green,0,899,us,70,1.634000,28.0,mww,171.371,-38.1911,-70.3731,"Loncopué, Argentina",South America,Argentina
3,"M 7.2 - 98 km S of Sand Point, Alaska",7.2,16-07-2023 06:48,6,6,green,1,860,us,173,0.907000,36.0,mww,32.571,54.3844,-160.6990,"Sand Point, Alaska",NaN,NaN
4,M 7.3 - Alaska Peninsula,7.3,16-07-2023 06:48,0,5,NaN,1,820,at,79,0.879451,172.8,Mi,21.000,54.4900,-160.7960,Alaska Peninsula,NaN,NaN


##  Preprocessing

In [5]:
df['latitude'] = df['latitude'].round(2)
df['longitude'] = df['longitude'].round(2)      

### Checking for Null


#### - Handling Location Nulls


In [6]:
df_loc_na = df[df['location'].isna()]
df_loc_na

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
30,M 6.8 -,6.8,20-01-2023 22:09,4,3,green,0,718,us,60,5.129,61.0,mww,610.671,-26.72,-63.04,NaN,South America,Argentina
37,M 7.0 -,7.0,12-11-2022 07:09,3,3,green,1,755,us,147,3.125,18.0,mww,579.000,-20.05,-178.35,NaN,Oceania,Fiji
39,M 6.6 -,6.6,09-11-2022 10:14,0,2,green,1,670,us,131,4.998,27.0,mww,624.464,-25.59,178.28,NaN,NaN,NaN
54,M 6.9 -,6.9,19-05-2022 10:13,2,5,green,1,733,us,127,0.371,45.0,mww,10.000,-54.13,159.03,NaN,NaN,NaN
103,M 6.9 -,6.9,01-05-2021 01:27,7,6,green,1,919,us,0,2.619,35.0,mww,43.000,38.23,141.66,NaN,Asia,Japan
281,M 6.9 -,6.9,28-05-2016 05:38,3,3,green,1,733,us,0,5.485,19.0,mww,405.690,-21.97,-178.20,NaN,NaN,NaN


In [7]:
from geopy.geocoders import Nominatim
import time

In [8]:
# Filling na values in location column using geopy 
def impute_locations(df, lat_col='latitude', lon_col='longitude', location_col='location'):
    """
    Impute missing location values with English names of country and state/province.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing latitude, longitude, and location columns
    lat_col (str): Name of the latitude column
    lon_col (str): Name of the longitude column
    location_col (str): Name of the location column to impute
    
    Returns:
    pandas.DataFrame: DataFrame with imputed location values in English
    """
    df_copy = df.copy()
    # Set language preference to English
    geolocator = Nominatim(user_agent="my_agent")
    
    def get_location(row):
        if pd.isna(row[location_col]):
            try:
                coords = f"{row[lat_col]}, {row[lon_col]}"
                # Request data in English
                location = geolocator.reverse(coords, language='en')
                
                if location and location.raw.get('address'):
                    address = location.raw['address']
                    # Get English names for state/province and country
                    state = (
                        address.get('state_en') or 
                        address.get('state') or 
                        address.get('province_en') or 
                        address.get('province') or 
                        ''
                    )
                    country = address.get('country_en') or address.get('country', '')
                    return f"{state}, {country}".strip(', ')
                return None
            except Exception as e:
                print(f"Error getting location for coordinates {coords}: {e}")
                return None
            finally:
                time.sleep(1)
        return row[location_col]
    
    mask = df_copy[location_col].isna()
    if mask.any():
        print(f"Imputing {mask.sum()} missing locations...")
        df_copy[location_col] = df_copy.apply(get_location, axis=1)
    
    return df_copy


In [9]:
df = impute_locations(df)
print("\nResults:")
df.isna().sum()

Imputing 6 missing locations...

Results:


title          0
magnitude      0
date_time      0
cdi            0
mmi            0
alert        551
tsunami        0
sig            0
net            0
nst            0
dmin           0
gap            0
magType        0
depth          0
latitude       0
longitude      0
location       3
continent    716
country      349
dtype: int64

In [10]:
print(df.iloc[37])

title                M 7.0 - 
magnitude                 7.0
date_time    12-11-2022 07:09
cdi                         3
mmi                         3
alert                   green
tsunami                     1
sig                       755
net                        us
nst                       147
dmin                    3.125
gap                      18.0
magType                   mww
depth                   579.0
latitude               -20.05
longitude             -178.35
location        Eastern, Fiji
continent             Oceania
country                  Fiji
Name: 37, dtype: object


In [11]:
df['location'] = df['location'].replace('None', np.nan)
df = df.dropna(subset=['location'])

#### - Handling Country Null

In [12]:
# Extracting values for country column from location column
df['country'] = df['country'].fillna(df['location'].str.split(',').str[1])

In [13]:
df.isna().sum()

title          0
magnitude      0
date_time      0
cdi            0
mmi            0
alert        551
tsunami        0
sig            0
net            0
nst            0
dmin           0
gap            0
magType        0
depth          0
latitude       0
longitude      0
location       0
continent    713
country       51
dtype: int64

In [14]:
df_country_na = df[df['country'].isna()]
df_country_na['location'].unique()

array(['Alaska Peninsula', 'Tonga', 'the Fiji Islands',
       'the Loyalty Islands', 'Kermadec Islands region',
       'the Kermadec Islands', 'Vanuatu region',
       'South Sandwich Islands region', 'Mauritius - Reunion region',
       'central Mid-Atlantic Ridge', 'Prince Edward Islands region',
       'Bouvet Island region', 'Fiji region', 'Micronesia region',
       'Okhotsk', 'northern Mid-Atlantic Ridge',
       'off the west coast of northern Sumatra',
       'Philippine Islands region', 'the Kuril Islands', 'Kuril Islands',
       'Macquarie Island', 'South Indian Ocean'], dtype=object)

In [15]:
# Filling in the remaining null values in country column using geopy module 
from geopy.exc import GeocoderTimedOut

def get_country_from_location(df, location_col='location', country_col='country'):
    """
    Convert location names to country names using geocoding.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing location column
    location_col (str): Name of the location column
    country_col (str): Name of the country column to fill
    
    Returns:
    pandas.DataFrame: DataFrame with filled country values
    """
    df_copy = df.copy()
    geolocator = Nominatim(user_agent="my_geocoder")
    
    def extract_country(location):
        if pd.isna(location):
            return None
            
        try:
            # Search for the location
            geocode_result = geolocator.geocode(
                location,
                language='en',  # Get results in English
                addressdetails=True,  # Get detailed address information
                exactly_one=True  # Return only the best match
            )
            
            if geocode_result and geocode_result.raw.get('address'):
                # Get country from address details
                country = geocode_result.raw['address'].get('country')
                return country
                
        except GeocoderTimedOut:
            print(f"Timeout for location: {location}")
        except Exception as e:
            print(f"Error processing location '{location}': {e}")
            
        time.sleep(1)  # Respect API rate limits
        return None
    
    # Only process rows where country is missing
    mask = df_copy[country_col].isna()
    if mask.any():
        print(f"Processing {mask.sum()} missing country values...")
        df_copy.loc[mask, country_col] = df_copy.loc[mask, location_col].apply(extract_country)
        
        # Print results
        filled = mask.sum() - df_copy[country_col].isna().sum()
        print(f"\nFilled {filled} out of {mask.sum()} missing country values")
        
        if df_copy[country_col].isna().sum() > 0:
            print("\nLocations that couldn't be mapped:")
            print(df_copy[df_copy[country_col].isna()][location_col].unique())
    
    return df_copy


In [16]:
df = get_country_from_location(df)


Processing 51 missing country values...

Filled 28 out of 51 missing country values

Locations that couldn't be mapped:
['Kermadec Islands region' 'the Kermadec Islands'
 'South Sandwich Islands region' 'central Mid-Atlantic Ridge'
 'Prince Edward Islands region' 'Bouvet Island region'
 'northern Mid-Atlantic Ridge' 'off the west coast of northern Sumatra'
 'the Kuril Islands']


In [17]:
df["country"].unique()

array(['Vanuatu', ' El Salvador', 'Argentina', ' Alaska', 'United States',
       ' Antigua and Barbuda', 'Tonga', 'Fiji', 'Colombia', 'France',
       ' Tonga', 'Indonesia', ' New Zealand', 'Russian Federation (the)',
       'Papua New Guinea', 'Afghanistan', 'Ecuador', None, ' Vanuatu',
       'Tajikistan', 'Turkiye', 'Solomon Islands', ' Indonesia', 'Panama',
       'Mexico', 'Taiwan', "People's Republic of China", 'Philippines',
       'Brazil', 'Peru', ' Nicaragua', ' New Caledonia', ' Japan',
       ' Cyprus', 'United States of America', 'Antarctica', 'Haiti',
       ' Philippines', ' Panama', ' Wallis and Futuna', 'Japan',
       ' Russia', 'Mongolia', ' Chile', 'Greece', 'Chile',
       ' Solomon Islands', ' Greece', 'Russia', ' Jamaica', 'Turkey',
       ' Fiji',
       'United Kingdom of Great Britain and Northern Ireland (the)',
       ' Australia', ' South Sandwich Islands',
       'South Georgia and the South Sandwich Islands',
       ' Svalbard and Jan Mayen', ' Canada', 

In [18]:
# Removing trail spaces in country column
df["country"] = df["country"].str.strip()

In [19]:
# Removing the word "region" from values in country column
df['country'] = df['country'].str.replace(r'region', '', regex=True)

In [20]:
df.dropna(subset="country",axis =0,inplace = True)

In [21]:
# Converting U.S. state names in the country column to 'United States of America'.
def convert_states_to_usa(df, country_col='country'):
    """
    Convert U.S. state names in the country column to 'United States of America'.

    Parameters:
    df (pandas.DataFrame): DataFrame containing the country column
    country_col (str): Name of the column to update with 'United States of America'

    Returns:
    pandas.DataFrame: Updated DataFrame
    """
    # List of U.S. states and territories
    us_states_and_territories = [
        'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
        'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
        'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
        'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
        'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
        'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
        'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
        'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
        'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
        'West Virginia', 'Wisconsin', 'Wyoming',
        # U.S. territories
        'District of Columbia', 'Puerto Rico', 'Guam', 'American Samoa',
        'U.S. Virgin Islands', 'Northern Mariana Islands'
    ]

    # Replace state names in the country column
    df[country_col] = df[country_col].apply(
        lambda x: 'United States of America' if x in us_states_and_territories else x
    )

    return df


# Apply the function
df = convert_states_to_usa(df, country_col='country')

print(df)


                                         title  magnitude         date_time  \
0             M 6.5 - 42 km W of Sola, Vanuatu        6.5  16-08-2023 12:47   
1     M 6.5 - 43 km S of Intipucá, El Salvador        6.5  19-07-2023 00:22   
2     M 6.6 - 25 km ESE of Loncopué, Argentina        6.6  17-07-2023 03:05   
3        M 7.2 - 98 km S of Sand Point, Alaska        7.2  16-07-2023 06:48   
4                     M 7.3 - Alaska Peninsula        7.3  16-07-2023 06:48   
..                                         ...        ...               ...   
995          M 7.1 - 85 km S of Tungor, Russia        7.1  27-05-1995 13:03   
996     M 7.7 - 249 km E of Vao, New Caledonia        7.7  16-05-1995 20:12   
997  M 6.9 - 27 km NNW of Maubara, Timor Leste        6.9  14-05-1995 11:33   
998           M 6.6 - 10 km W of Aianí, Greece        6.6  13-05-1995 08:47   
999  M 7.1 - 14 km NE of Cabatuan, Philippines        7.1  05-05-1995 03:53   

     cdi  mmi   alert  tsunami  sig net  nst      d

In [22]:
df.to_csv("0_earthquake_1995-2023.csv")